<a href="https://colab.research.google.com/github/TI1307/DM_Project/blob/main/notebooks/cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Data Preprocessing and cleaning


# Data Quality **Issues**


In [ ]:
import pandas as pd
import matplotlib_inline
from datetime import datetime
import unicodedata
import re


In [ ]:
########    LIBRARY INVENTORY TABLE      ###############
LIBRARY_INVENTORY_REAL= pd.read_excel("../data/LIBRARY INVENTORY.xlsx")
print ('Number of instance in the LIBRARY INVENTORY' , (LIBRARY_INVENTORY_REAL.shape[0]))
print ('Number of attributes in the LIBRARY INVENTORY' , (LIBRARY_INVENTORY_REAL.shape[1]))
print ('-'*20)
########  LIBRARY_CARALOGUE TABLE     ###############
LIBRARY_CATALOGUE_REAL= pd.read_excel("../data/library catalogue.xlsx")
print ('Number of instance in the LIBRARY CARALOGUE' , (LIBRARY_CATALOGUE_REAL.shape[0]))
print ('Number of attributes in the LIBRARY INVENTORY' , (LIBRARY_CATALOGUE_REAL.shape[1]))
print ('-'*20)
#######    BORROWING TABLE             ###########################
BORROWINGS_REAL= pd.read_excel("../data/Borrowings .xlsx")
print ('Number of instance in the BORROWINGS TABLE ' , (BORROWINGS_REAL.shape[0]))
print ('Number of attributes in the BORROWINGS TABLE ' , (BORROWINGS_REAL.shape[1]))


### Data Copy


In [ ]:
LIBRARY_INVENTORY =LIBRARY_INVENTORY_REAL.copy()
LIBRARY_CATALOGUE =LIBRARY_CATALOGUE_REAL.copy()
BORROWINGS =BORROWINGS_REAL.copy()




## **Cleaning Functions**


### **Missing Values**


In [ ]:
# gives a summery of missing values
def missing_values_summary(df):
  missing_count = df.isnull().sum()
  missing_percent = (missing_count/ len(df))*100

  summery = pd.DataFrame({
      'missing_count': missing_count,
      'missing_percent': missing_percent
  })
  summery = summery.sort_values('missing_percent', ascending=False)
  return summery
import pandas as pd

def handle_missing_values(df, strategy_dict):
    """
    Fill missing values in a DataFrame according to a strategy dictionary.

    Parameters:
        df (pd.DataFrame): Input dataframe
        strategy_dict (dict): {column_name: strategy}, where strategy can be:
            - 'mean'   → fill numeric with mean
            - 'median' → fill numeric with median
            - 'mode'   → fill categorical with mode
            - value    → fill with this value directly

    Returns:
        pd.DataFrame: DataFrame with missing values handled
    """
    for col, strategy in strategy_dict.items():
        if strategy == 'mean':
            df[col] = df[col].fillna(df[col].mean())
        elif strategy == 'median':
            df[col] = df[col].fillna(df[col].median())
        elif strategy == 'mode':
            df[col] = df[col].fillna(df[col].mode()[0])
        else:
            # Use the provided value
            df[col] = df[col].fillna(strategy)
    return df




### **Duplicate Data**


In [ ]:
def checkDuplicat (df, table_name,  keep=False):
    dup=df.duplicated()
    dup_count=dup.sum()
    print(f'Number of duplicate rows in {table_name} table is = %d' % (dup_count))
    if dup_count > 0:
      print (f'\n Duplicare row in the table {table_name}:')
      dup_rows=df[dup]
      for idx in dup_rows.index:
          print(f'\n row number :{idx}')
          print(df.loc[idx])
          print('-' * 50)




### Feature Engineering


In [ ]:
# Dates formating 
def format_dates(df, date_columns):
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    return df

def borrowing_duration(df,Date_prêt,Date_retour):
    format_dates(df, [Date_prêt,Date_retour])
    df['borrowing duration']=(df[Date_retour]-df[Date_prêt]).dt.days
    return 
# Clean titles (remove extra spaces, lowercase for matching)

def clean_title(title):
    """Normalize title for matching"""
    if pd.isna(title):
        return None
    title = str(title).strip().upper()
    title = unicodedata.normalize('NFKD', title)
    title = ''.join(c for c in title if not unicodedata.combining(c))
    title = re.sub(r'[^\w\s]', ' ', title)
    title = ' '.join(title.split())
    return title
# Normalize titles
BORROWINGS['Titre_clean'] = BORROWINGS['Titre'].apply(clean_title) 
LIBRARY_CATALOGUE['Titre_clean'] = LIBRARY_CATALOGUE['Titre'].apply(clean_title)
LIBRARY_INVENTORY['Titre_clean'] = LIBRARY_INVENTORY['Titre'].apply(clean_title)



# **Borowing Table**


**Table info**


In [ ]:
# Cleaningg the borrowing table
# table name : BORROWINGS

print ('Number of instance in the BORROWINGS TABLE ' , (BORROWINGS.shape[0]))
print ('Number of attributes in the BORROWINGS TABLE ' , (BORROWINGS.shape[1]))
print(BORROWINGS.columns)

**Missing Values & Check Duplicate**


In [ ]:
# summery about missing values :
print(missing_values_summary(BORROWINGS))

#check duplicate :
checkDuplicat(BORROWINGS , "BORROWINGS TABLE ")

**Hundle Missing Values**


In [ ]:
# we drop Date réservation because more then 60% (98%) of its value is missing 
BORROWINGS=BORROWINGS.drop(['Date réservation'],axis=1) 

**Outliers**:

First we convert the dates from string to date type , then calculate borrowing duration ( Date_retour- date_pret) , to get numurical values so we can detect outliers using box blot


In [ ]:
borrowing_duration(BORROWINGS,'Date prêt','Date retour')
BORROWINGS = BORROWINGS.drop(['Date prêt', 'Date retour'], axis=1)
print(BORROWINGS.head())

In [ ]:
BORROWINGS.boxplot(column='borrowing duration')

In [ ]:
BORROWINGS[BORROWINGS['borrowing duration']>250]

# **LIBRARY INVENTORY**


**Table info**


In [ ]:
# Cleaningg the LIBRARY INVENTORY table
# table name : LIBRARY_INVENTORY

print ('Number of instance in the LIBRARY_INVENTORY TABLE ' , (LIBRARY_INVENTORY.shape[0]))
print ('Number of attributes in the LIBRARY_INVENTORY TABLE ' , (LIBRARY_INVENTORY.shape[1]))
print(LIBRARY_INVENTORY.columns)

**Missing Values & Check Duplicate**


In [ ]:
# summery about missing values :
print(missing_values_summary(LIBRARY_INVENTORY))

#check duplicate :
checkDuplicat(LIBRARY_INVENTORY, "LIBRARY_INVENTORY")

Here we only have the columns : " Date d'édition " , " Auteur " , " Cote " , "Localisation" , " Statut " , " N° inventaire " , " Titre ","Mode d'acquisition" . we can use theme the others are more then 60% empty


In [ ]:
LIBRARY_INVENTORY = LIBRARY_INVENTORY.loc[:, (LIBRARY_INVENTORY.isnull().sum() / len(LIBRARY_INVENTORY)) * 100 <= 60]

**Outliers**


In [ ]:


LIBRARY_INVENTORY["Date d'édition"] = pd.to_numeric(LIBRARY_INVENTORY["Date d'édition"], errors="coerce")
LIBRARY_INVENTORY.boxplot(column=["Date d'édition"])

In [ ]:
LIBRARY_INVENTORY[LIBRARY_INVENTORY["Date d'édition"]>10000]

Those are Prototypes , so we need to handle them with a general rule , we will assume that Valid publication years are between 1000 and current year . we replace the nonvalid year , with the mode year per title , else we make it Nan


In [ ]:
# Define valid year range
current_year = datetime.now().year
invalid_mask = (
    (LIBRARY_INVENTORY["Date d'édition"] < 1000) |
    (LIBRARY_INVENTORY["Date d'édition"] > current_year)
)
# Compute MODE year per title (ignoring invalid years)
title_year_mode = (
    LIBRARY_INVENTORY.loc[~invalid_mask]
    .groupby("Titre_clean")["Date d'édition"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA)
)

# Replace invalid years using title mode
LIBRARY_INVENTORY.loc[invalid_mask, "Date d'édition"] = (
    LIBRARY_INVENTORY.loc[invalid_mask, "Titre_clean"]
    .map(title_year_mode)
)

# **LIBRARY_CATALOGUE**


**table info**


In [ ]:
# Cleaningg the LIBRARY_CATALOGUE table
# table name : LIBRARY_CATALOGUE

print ('Number of instance in the LIBRARY_CATALOGUE TABLE ' , (LIBRARY_CATALOGUE.shape[0]))
print ('Number of attributes in the LIBRARY_CATALOGUE TABLE ' , (LIBRARY_CATALOGUE.shape[1]))
print(LIBRARY_CATALOGUE.columns)

**Missing Values & check Duplicate**


In [ ]:
# summery about missing values :
print(missing_values_summary(LIBRARY_CATALOGUE))

#check duplicate :
checkDuplicat(LIBRARY_CATALOGUE, "LIBRARY_CATALOGUE")

all of those columns we can not hundle the missing values , except for the cote , we can use later some model to predict the category from the title , this will help us in clustering


**Outliers**


In [ ]:
LIBRARY_CATALOGUE.boxplot(column='Nbr. Exp.')

In [ ]:
LIBRARY_CATALOGUE[LIBRARY_CATALOGUE['Nbr. Exp.']>120]

**_save individual cleaned datasets_**


In [ ]:
LIBRARY_CATALOGUE.to_excel("../Clean_Data/cleaned_catalogue.xlsx", index=False)
BORROWINGS.to_excel("../Clean_Data/cleaned_borrowings.xlsx", index=False)
LIBRARY_INVENTORY.to_excel("../Clean_Data/cleaned_inventory.xlsx", index=False)
